In [1]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import psycopg2 as pg2
import random
%matplotlib inline

In [2]:
user = ''
password = ''
port = ''

def get_gdf_from_postgis(query):
    con = pg2.connect(host = 'localhost', database = 'kansai_osm', user = user, password = password, port = port)
    cur = con.cursor()

    df = gpd.read_postgis(query, con, geom_col = 'way', crs = 3857)
    df = df.to_crs(epsg=4326)

    con.close()
    cur.close()
    
    return df

#### Mapping Lines

Plotting out thousand of the longest expressways in this region. Equivalence between road types in japan and OSM highway key values can be found [here](https://wiki.openstreetmap.org/wiki/Highway:International_equivalence). OSM motorways correspond to expressways in Japan. Long roads are typically broken into segments in OSM however, so it's more correct to say the longest segments of expressways.

In [3]:
query = """SELECT osm_id, name, way, ST_Length(way) as path_length, tags -> 'maxspeed' as maxspeed
        FROM planet_osm_line
        WHERE highway = 'motorway'
        ORDER BY ST_Length(way) DESC
        LIMIT 1000;"""
motorways_df = get_gdf_from_postgis(query)
motorways_df.head()

,osm_id,name,way,path_length,maxspeed
0,33407955,阪神高速3号神戸線,"LINESTRING (135.3814043 34.71669139963393, 135...",21440.040938,None
1,33434021,阪神高速4号湾岸線,"LINESTRING (135.4213767 34.52206369962335, 135...",21211.131556,80
2,66794186,名阪国道,"LINESTRING (135.8857554 34.61993689962851, 135...",20723.113834,None
3,256421957,近畿自動車道,"LINESTRING (135.5755604 34.59324739962708, 135...",17128.940689,80
4,764092509,近畿自動車道,"LINESTRING (135.5922107 34.70842519963345, 135...",16620.716262,80


In [4]:
map_center = list(reversed(list(motorways_df.iloc[0]['way'].centroid.coords[0])))
folium_map = folium.Map(height = 550, width = 950, location = map_center, tiles = "CartoDB positron", 
                        attr = "Stamen", zoom_start = 8)

for i, row in motorways_df.iterrows():
    motorway_name = row['name']
    motorway_maxspeed = row['maxspeed']
    if row['name'] is None:
        motorway_name = "(unknown)"
    if row['maxspeed'] is None:
        motorway_maxspeed = "N/A"
        
    tooltip_text = motorway_name + "({}m length, ".format(round(row['path_length'], 2))
    tooltip_text += "{} speed limit)".format(motorway_maxspeed)
    
    points = [(i[1], i[0]) for i in row['way'].coords]
    folium.PolyLine(locations = points, tooltip = tooltip_text, popup = tooltip_text,
                  color = '#55DD88').add_to(folium_map)
    
folium_map